# Perceptron and Multi-Layer Perceptron

First the imports, Always

In [5]:
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as pl
%matplotlib inline

## Artificial Neural Networks

There are many ways of knitting the nodes of a neural network together, and each way results in a more or less complex behavior. Possibly the simplest of all topologies is the feed-forward network. Signals flow in one direction only; there is never any loop in the signal paths.

![title](img/NNFL_1.png)


Typically, ANN’s have a layered structure. The input layer picks up the input signals and passes them on to the next layer, the so-called ‘hidden’ layer. (Actually, there may be more than one hidden layer in a neural network.) Last comes the output layer that delivers the result.

Note: Input layer is not a layer as such, and should not be called an input layer, rather, should be named only as "Inputs". However, since many of the textbooks do adhere to this notation, we use the term input layer here.

![title](img/NNFL_2.png)


Just like a biological neuron has dendrites to receive signals, a cell body to process them, and an axon to send signals out to other neurons, the artificial neuron has a number of input channels, a processing stage, and one output that can fan out to multiple other artificial neurons.

#### 1) Each input gets scaled up or down

When a signal comes in, it gets multiplied by a weight value that is assigned to this particular input. That is, if a neuron has three inputs, then it has three weights that can be adjusted individually. During the learning phase, the neural network can adjust the weights based on the error of the last test result.

#### 2) All signals are summed up

In the next step, the modified input signals are summed up to a single value. In this step, an offset is also added to the sum. This offset is called bias. The neural network also adjusts the bias during the learning phase.

This is where the magic happens! At the start, all the neurons have random weights and random biases. After each learning iteration, weights and biases are gradually shifted so that the next result is a bit closer to the desired output. This way, the neural network gradually moves towards a state where the desired patterns are “learned”.

#### 3) Activation

Finally, the result of the neuron’s calculation is turned into an output signal. This is done by feeding the result to an activation function (also called transfer function).

# The Perceptron

The most basic form of an activation function is a simple binary function that has only two possible results.

![title](img/NNFL_3.png)


This function returns 1 if the input is positive or zero, and 0 for any negative input. A neuron whose activation function is a function like this is called a perceptron.

![title](img/NNFL_4.png)


A single perceptron, as bare and simple as it might appear, is able to learn where this line is, and when it finished learning, it can tell whether a given point is above or below that line.

## Single Perceptron for a two input AND gate 

In [3]:
class Perceptron:
    def __init__(self,X,Y,eta,X_test):
        self.w=np.random.uniform(1,2,X.shape[1]+1) #one weight for the bias
        self.x=X
        self.y=Y
        self.e=eta
        Perceptron.Train(self)
        Perceptron.Test(self,X_test)
    
    def Train(self):
        x_fin=np.array([np.ones(self.x.shape[0])])
        self.x=np.concatenate((self.x,np.transpose(x_fin)),axis=1)
        for i in range(100):
            y_pred=np.dot(self.x,np.transpose(self.w))
            y_pred[y_pred>=0]=1
            y_pred[y_pred<0]=-1
            err=self.y-y_pred
            for j in range(3):
                self.w=self.w + self.e*err[j]*self.x[j]

    
    def Test(self,X_test):
        x_bias=np.array([np.ones(X_test.shape[0])])
        X_test=np.concatenate((X_test,np.transpose(x_bias)),axis=1)
        y_test_pred=np.dot(X_test,np.transpose(self.w))
        y_test_pred[y_test_pred>=0]=1
        y_test_pred[y_test_pred<0]=-1
        print(y_test_pred)
        

X=np.array([[-1,-1],[-1,1],[1,-1],[1,1]])
Y=np.array([-1,-1,-1,1])
X_test=np.array([[-1,-1],[-1,1],[1,-1],[1,1]])
Perceptron(X,Y,0.01,X_test)



(3,)
[-1. -1. -1.  1.]


## Now lets try the same thing with a XOR Gate using MLP

Let's initialize the dataset

In [6]:
#Input array
X=np.array([[1,0,1,0],[1,0,1,1],[0,1,0,1],[0,0,0,1]])

#Output
y=np.array([[0],[1],[0],[1]])


# Multi-Layered Perceptron

As we can see the XOR Gate fails on a single perceptron. A look at the graph will give us enough intuition as to why this is the case.

![title](img/xor.png)

 We need two lines instead of the one generated by the perceptron. Since we obviously can't use a single perceptron for this case we have to make our architecture slightly more complex. We'll use the backpropagation algorithm. This has two phases :
 
 1) <b>Forward Propagation</b> (Exactly what happens during a normal perceptron)
 
 2) <b>Backpropagation</b> (Calculating the errors and updating weights. This differs between layers)

![title](img/backprop1.png)

![title](img/backprop2.png)



### Sigmoid Function (Activation)

Lets make the activation function that we'll be using. This activation function is called sigmoid and is extremely useful because of one particular property that makes it valuable during backpropagation. It also squashes the output between 0 and 1 making it easier for us to calculate probabilities for the output signals.
#### This is the equation
![title](img/sigmoid_equation.png)


![title](img/sigmoid_curve.png)


We see that when we diffrentiate this equation we get a very simple equation :
![title](img/sigmoid_derivative.jpg)
 

Using this during the backpropagation phase of the algorithm is computationally less intensive than calculating other gradients.


In [7]:
#Sigmoid Function
def sigmoid (x):
    return 1/(1 + np.exp(-x))

#Derivative of Sigmoid Function
def derivatives_sigmoid(x):
    return x * (1 - x)

## Assembling the MLP together

### First initialize all the variables and set the hyperparameters

In [15]:
#Variable initialization
epoch=5000 #Setting training iterations
lr=0.1 #Setting learning rate
inputlayer_neurons = X.shape[1] #number of features in data set
hiddenlayer_neurons = 3 #number of hidden layers neurons
output_neurons = 1 #number of neurons at output layer

#weight and bias initialization
np.random.seed(420)
wh=np.random.uniform(size=(inputlayer_neurons,hiddenlayer_neurons))
bh=np.random.uniform(size=(1,hiddenlayer_neurons))
wout=np.random.uniform(size=(hiddenlayer_neurons,output_neurons))
bout=np.random.uniform(size=(1,output_neurons))

### Now for the actual algorithm

In [16]:
for i in range(epoch):

    #Forward Propogation
    hidden_layer_input1=np.dot(X,wh)
    hidden_layer_input=hidden_layer_input1 + bh
    hiddenlayer_activations = sigmoid(hidden_layer_input)
    output_layer_input1=np.dot(hiddenlayer_activations,wout)
    output_layer_input= output_layer_input1+ bout
    output = sigmoid(output_layer_input)

    #Backpropagation

    Error = y - output
    
    # Slopes are found using derivatives of its output
    slope_output_layer = derivatives_sigmoid(output)
    slope_hidden_layer = derivatives_sigmoid(hiddenlayer_activations)
    
    # Found using multiplication of Error and ____
    delta_output = Error * slope_output_layer
    
    
    Error_at_hidden_layer = delta_output.dot(wout.T)
    
    # Similar to delta_output
    delta_hidden_layer = Error_at_hidden_layer * slope_hidden_layer
    
    # Weight and Bias Update
    wout += hiddenlayer_activations.T.dot(delta_output)*lr
    bout += np.sum(delta_output, axis=0,keepdims=True) *lr
    wh += X.T.dot(delta_hidden_layer)*lr
    bh += np.sum(delta_hidden_layer, axis=0,keepdims=True) *lr

print (output)
 

[[ 0.04268954]
 [ 0.96073561]
 [ 0.03243673]
 [ 0.99317504]]
